In [1]:
import gensim, logging

C:\Users\asus\AppData\Local\Continuum\anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [69]:
from gensim import corpora, models, similarities

C:\Users\asus\AppData\Local\Continuum\anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [93]:
import os
import re
from collections import defaultdict

Сначала создаём корпус текстов.

In [57]:
#Так мы создаём список текстов, которые будут векторизоваться (простите за этот костыльный метод)
files = []
for i, file in enumerate(os.listdir('C://Users//asus//Desktop//Cybercorpora//Toys')):
    jj = os.path.abspath('Desktop\\Cybercorpora\\Toys\\' + file)
    dd = re.sub(r'\\', '//', jj)
    xx = open(jj, 'r', encoding = 'utf-8')
    ii = xx.read()
    files.append(ii)


In [64]:
#токенизируем. Потом можно сделать стоп-лист и вообще почистить тексты, но пока оставим такой игрушечный пример
texts = [[word for word in file.lower().split()] for file in files]
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

texts = [[token for token in text if frequency[token] > 1] for text in texts]

In [75]:
#векторизуем
dictionary = corpora.Dictionary(texts)
dictionary.save('C:/Users/asus/Desktop/Cybercorpora/Vectors/corpus.dict')
print(dictionary)

Dictionary(754 unique tokens: ['степени', 'тесно', 'внешней', 'состоит', 'religious']...)


In [78]:
corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize('C:/Users/asus/Desktop/Cybercorpora/Vectors/corpus.mm', corpus)

In [80]:
#Переходим к сходству текстов
dictionary = corpora.Dictionary.load('C:/Users/asus/Desktop/Cybercorpora/Vectors/corpus.dict')
corpus = corpora.MmCorpus('C:/Users/asus/Desktop/Cybercorpora/Vectors/corpus.mm')
print(corpus)

MmCorpus(4 documents, 754 features, 1065 non-zero entries)


In [81]:
lsi = models.LsiModel(corpus, id2word=dictionary, num_topics=2)

In [84]:
#тут я считаю сходство
doc = "Human computer interaction"
vec_bow = dictionary.doc2bow(doc.lower().split())
vec_lsi = lsi[vec_bow]
print(vec_lsi)


[(0, 0.016355223081294414), (1, -0.05094541097829459)]


In [86]:
#Никакого сходства, что разумно - другой язык
doc = "Ich hasse Sprachwissenschaft"
vec_bow = dictionary.doc2bow(doc.lower().split())
vec_lsi = lsi[vec_bow]
print(vec_lsi)

[]


In [91]:
#Попробуем подать политологический текст
doc = open('C:/Users/asus/Desktop/Cybercorpora/Toys/textstocompare/abhaziya-na-perekrestke-geopoliticheskih-interesov.txt', 'r', encoding = 'utf-8').read()
vec_bow = dictionary.doc2bow(doc.lower().split())
vec_lsi = lsi[vec_bow]
print(vec_lsi)

[(0, 18.725760398105304), (1, 3.3837635750276047)]


In [92]:
#А теперь из педагогики
doc = open('C:/Users/asus/Desktop/Cybercorpora/Toys/textstocompare/adaptivnost-i-kreativnost-v-stanovlenii-i-razvitii-tvorcheskoy-lichnosti-buduschego-uchitelya-v-sisteme-nepreryvnogo-pedagogicheskogo.txt', 'r', encoding = 'utf-8').read()
vec_bow = dictionary.doc2bow(doc.lower().split())
vec_lsi = lsi[vec_bow]
print(vec_lsi)

[(0, 119.34819809284963), (1, -9.592354497226992)]


Что-то он считает. Но что? 